<a href="https://colab.research.google.com/github/a12dongithub/DeepFake-Detection/blob/main/Deepfake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 ! pip install -q kaggle
 from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"a12donkaggle","key":"31e91be97791d2f84b7496fcb8a153fd"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list
!kaggle datasets list -s face
!kaggle datasets download -d sorokin/faceforensics
!unzip faceforensics.zip -d faceforensic/

Streaming output truncated to the last 5000 lines.
  inflating: faceforensic/manipulated_sequences/Deepfakes/c23/videos/000_003.mp4  
  inflating: faceforensic/manipulated_sequences/Deepfakes/c23/videos/001_870.mp4  
  inflating: faceforensic/manipulated_sequences/Deepfakes/c23/videos/002_006.mp4  
  inflating: faceforensic/manipulated_sequences/Deepfakes/c23/videos/003_000.mp4  
  inflating: faceforensic/manipulated_sequences/Deepfakes/c23/videos/004_982.mp4  
  inflating: faceforensic/manipulated_sequences/Deepfakes/c23/videos/005_010.mp4  
  inflating: faceforensic/manipulated_sequences/Deepfakes/c23/videos/006_002.mp4  
  inflating: faceforensic/manipulated_sequences/Deepfakes/c23/videos/007_132.mp4  
  inflating: faceforensic/manipulated_sequences/Deepfakes/c23/videos/008_990.mp4  
  inflating: faceforensic/manipulated_sequences/Deepfakes/c23/videos/009_027.mp4  
  inflating: faceforensic/manipulated_sequences/Deepfakes/c23/videos/010_005.mp4  
  inflating: faceforensic/manipulate

In [ ]:
!pip install efficientnet

     |████████████████████████████████| 51kB 8.0MB/s 


In [ ]:
import tensorflow as tf
import dlib
import cv2
import os
import numpy as np
from PIL import Image, ImageChops, ImageEnhance
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import pandas as pd
from efficientnet.tfkeras import EfficientNetB4
import dlib
import cv2
from PIL import Image, ImageChops, ImageEnhance
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import argparse
import time
import json
import cv2
import os
import dlib
import imutils
import pandas as pd
import numpy as np
from imutils import face_utils
from imutils.video import FileVideoStream, VideoStream
from scipy.spatial import distance as dist

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]
def get_video_length(video):
    cap = cv2.VideoCapture(video)
    fps = cap.get(cv2.CAP_PROP_FPS)      # OpenCV2 version 2 used "CV_CAP_PROP_FPS"
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count/fps
    cap.release()
    return duration
  
real = pd.read_csv('/content/drive/MyDrive/deepfakefile/blinking_Test_real_500.csv')
fake = pd.read_csv('/content/drive/MyDrive/deepfakefile/blinking_Test_fake_500.csv')

X=[]
Y=[]
for idx,rows in real.iterrows():
  path = '/content/faceforensic/original_sequences/youtube/c23/videos/' + str(rows['video name'])
  X.append(rows['total']/get_video_length(path))
  Y.append(0)

for idx,rows in fake.iterrows():
  path = '/content/faceforensic/manipulated_sequences/Deepfakes/c23/videos/' + str(rows['video name'])
  X.append(rows['total']/get_video_length(path))
  Y.append(1)

X=np.asarray(X)
Y=np.asarray(Y)

X = X.reshape((-1,1))
Y = Y.reshape((-1,1))

X,Y = unison_shuffled_copies(X,Y)

clf = DecisionTreeClassifier(random_state=0).fit(X, Y)
clf.score(X, Y)

0.8948919449901768

In [ ]:
clf.predict_proba([[0.0]])[0][1]

0.7322175732217573

In [ ]:
model = load_model('/content/drive/MyDrive/deepfakefile/model_b3_300_4.h5')

input_shape = (300, 300, 3)
pr_data = []
detector = dlib.get_frontal_face_detector()
def go(path):
    cap = cv2.VideoCapture(str(path))
    frameRate = cap.get(5)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    if(height*width < 1000*700):
        return -1,-1,-1
    f =0
    r = 0
    thr = 0
    tmp = 0
    temp = 0
    while cap.isOpened():
        frameId = cap.get(1)
        ret, frame = cap.read()
        if ret != True:
            break
        if frameId % ((int(frameRate)+1)*1) == 0:
            face_rects, scores, idx = detector.run(frame, 0)
            for i, d in enumerate(face_rects):
                x1 = d.left()
                y1 = d.top()
                x2 = d.right()
                y2 = d.bottom()
                crop_img = frame[y1:y2, x1:x2]
                data = img_to_array(cv2.resize(crop_img, (300, 300))).flatten()
                data = data.reshape(-1, 300, 300, 3)
                tmp = temp
                temp = model.predict(data)[0]
                if(tmp > 0.99 and temp > 0.99):
                  thr = 0.3
                print(temp)
                if(temp < 0.85):
                    r+=1
                else:
                    f+=1
    val = f/(f+2*r) + thr
    if (val > 1):  
      return (f,r,1)
    else:
      return (f,r,val)

def get_video_length(video):
    cap = cv2.VideoCapture(video)
    fps = cap.get(cv2.CAP_PROP_FPS)      # OpenCV2 version 2 used "CV_CAP_PROP_FPS"
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count/fps
    cap.release()
    return duration

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

predictor = dlib.shape_predictor('/content/drive/MyDrive/deepfakefile/shape_predictor_68_face_landmarks.dat')
def goo(path):
    EYE_AR_THRESH = 0.2
    EYE_AR_CONSEC_FRAMES = 3
    COUNTER = 0
    TOTAL = 0
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
    vs = FileVideoStream(str(path)).start()
    fileStream = True
    
    #fileStream = False
    try:
        while True:
            if fileStream and not vs.more():
                break
            frame = vs.read()
            #cv2.imshow("Frame", frame)
            try:
                frame = imutils.resize(frame, width=800)
            except:
                continue
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            rects = detector(gray, 0)
            for rect in rects:
                shape = predictor(gray, rect)
                shape = face_utils.shape_to_np(shape)
                leftEye = shape[lStart:lEnd]
                rightEye = shape[rStart:rEnd]
                leftEAR = eye_aspect_ratio(leftEye)
                rightEAR = eye_aspect_ratio(rightEye)
                ear = (leftEAR + rightEAR) / 2.0

                if ear < EYE_AR_THRESH:
                    COUNTER += 1

                else:
                    if COUNTER >= EYE_AR_CONSEC_FRAMES:
                        TOTAL += 1
                    COUNTER = 0
        print("ALL DONE")
        cv2.destroyAllWindows()
        vs.stop()
        if(TOTAL == 0):
            return -1
        else:
            return TOTAL/get_video_length(path)
    except:
        return -1


df = pd.DataFrame({'name':[],'fake':[],'real':[],'prediction':[], 'EAR':[], 'predictions':[], 'final':[]})

right = 0
wrong = 0
i=0
L = 0.7
THRESH = 0.5

for name in os.listdir('/content/faceforensic/original_sequences/youtube/c23/videos/'):
    print(name)
    if(i > 100):
        break
    try:
      f,r,val = go('/content/faceforensic/original_sequences/youtube/c23/videos/' + name)
      if(val == -1):
        continue
      v = goo('/content/faceforensic/original_sequences/youtube/c23/videos/' + name)
      print(val)
      if(v == -1):
          continue
      print(v)
      value = L*val + (1-L)*clf.predict_proba([[v]])[0][1]
      df.loc[len(df.index)] = [name,f,r,val,v,clf.predict_proba([[v]])[0][1],value]
      print(value)
      if(i%20 == 0):
        df.to_csv('/content/drive/MyDrive/deepfakefile/frame_efficient_ear.csv')
      if (value > THRESH):
          print("WRONG!")
          wrong+=1
          i+=1
      else:
          print("right")
          right+=1
          i+=1
    except:
      print("error at " + name)

i =0
for name in os.listdir('/content/faceforensic/manipulated_sequences/Deepfakes/c23/videos/'):
    print(name)
    if(i > 100):
        break
    try:
        f,r,val = go('/content/faceforensic/manipulated_sequences/Deepfakes/c23/videos/' + name)
        if(val == -1):
          continue
        v = goo('/content/faceforensic/manipulated_sequences/Deepfakes/c23/videos/' + name)
        if(v == -1):
            continue
        value = L*val + (1-L)*clf.predict_proba([[v]])[0][1]
        df.loc[len(df.index)] = [name,f,r,val,v,clf.predict_proba([[v]])[0][1],value]
        print(value)
        if(i%20 == 0):
          df.to_csv('/content/drive/MyDrive/deepfakefile/frame_efficient_ear.csv')
        if(val <= THRESH):
            print("WRONG!")
            wrong+=1
            i+=1
        else:
            print("right")
            right+=1
            i+=1
    except:
        print("error at " + name)
df.to_csv('/content/drive/MyDrive/deepfakefile/frame_efficient_ear.csv')
#print(right)
#print(wrong)
print( right/(right+wrong))

0.9108910891089109
